## Show dblk size
`ls -l /<node_id:ffffffffffff>/tag/sd/0/img/dblk/byte`

## Show dblk current record number
`ls -al /<node_id:ffffffffffff>/tag/sd/0/img/dblk/.recnum`

## Show dblk current last known record offset
`ls -l /<node_id:ffffffffffff>/tag/sd/0/img/dblk/.last_rec`

## Show dblk current last known SYNC record offset
`ls -l /<node_id:ffffffffffff>/tag/sd/0/img/dblk/.last_sync`

## `tail -f` dblk file
`tail -f /<node_id:ffffffffffff>/tag/sd/0/img/dblk/byte`

In [ ]:
from __future__ import print_function, absolute_import, division

import logging
import os
import sys
from collections import defaultdict, OrderedDict
from errno import ENOENT, ENODATA
from stat import S_IFDIR, S_IFLNK, S_IFREG
from sys import argv, exit, path
from time import time, sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify

sys.path.insert(0,"../tagfuse")
from tagfuse import *
sys.path.insert(0,"../tagnet")
from tagnet import *

if not hasattr(__builtins__, 'bytes'):
    bytes = str

In [ ]:
from fuse import FuseOSError

In [ ]:
import inspect

In [ ]:
radio = radio_start()

In [ ]:
tag_tree = TagFuseRootTree(radio)

In [ ]:
def LocateNode(tag_tree, path):
    path_list = path2list(path)
    # print(path_list)
    if (path == '/'):
        print('located root')
        return tag_tree, path_list
    return tag_tree.traverse(path_list, 0)

def DeleteNode(path, node):
    pass

def path2list(path):
    path = os.path.abspath(os.path.realpath(path))
    return path.split('/')[1:]

In [ ]:
path2list('/')

In [ ]:
path2list('/.test/')

## Get Poll Count

In [ ]:
path='/ffffffffffff/tag/poll/cnt'
handler, path_list = LocateNode(tag_tree, path)
print(handler)
print(path_list)

In [ ]:
handler.getattr(path_list, update=True)

## Get Poll Events

In [ ]:
path='/ffffffffffff/tag/poll/ev'
handler, path_list = LocateNode(tag_tree, path)
type(handler)

In [ ]:
handler.getattr(path_list, update=True)

## Read Dblk 

In [ ]:
STOP

In [ ]:
path='/ffffffffffff/tag/sd/0/dblk/byte'
handler, path_list = LocateNode(tag_tree, path)
type(handler)

In [ ]:
attrs = handler.getattr(path_list, update=True)
attrs

In [ ]:
handler.read(path_list, 1, 62452739)

In [ ]:
handler.unlink(path_list)

In [ ]:
attrs = handler.getattr(path_list, update=True)
eof_offset = attrs['st_size']-1024
array = handler.read(path_list, 1024, eof_offset)

In [ ]:
print(attrs)
print(eof_offset, len(array), hexlify(array[:24]))

In [ ]:
attrs = handler.getattr(path_list, update=True)
eof_offset = attrs['st_size']
while eof_offset <= attrs['st_size']:
    try:
        array = handler.read(path_list, 300, eof_offset)
        #print(len(array), hexlify(array[:24]))
        eof_offset += len(array)
    except FuseOSError:
        sleep(.3)
        attrs = handler.getattr(path_list, update=True)
        print('check for new eof', eof_offset, attrs['st_size'])

##  Echo Test 

In [ ]:
path='/.test/echo'
handler, path_list = LocateNode(tag_tree, path)
type(handler)
from binascii import hexlify

In [ ]:
type(handler)

In [ ]:
handler.sparse

In [ ]:
handler.write(path_list, '\x02' * 512, 0) + handler.write(path_list, '\x03' * 512, 512)

In [ ]:
b = handler.read(path_list, 512, 0)
print(len(b), hexlify(b))

In [ ]:
b = handler.read(path_list, 512, 512)
print(len(b), hexlify(b))

In [ ]:
try:
    b = handler.read(path_list, 512, 1024)
    print(len(b), hexlify(b))
except FuseOSError as e:
    print(e)

In [ ]:
b = handler.read(path_list, 16, 500)
print(len(b), hexlify(b))

In [ ]:
attrs = handler.getattr(path_list, update=True)
attrs

In [ ]:
len(handler.sparse)

In [ ]:
len(handler.sparse[0])

In [ ]:
for k,v in handler.sparse.items():
    print(k, len(v))

In [ ]:
handler.flush(path_list)

In [ ]:
handler.unlink(path_list)

In [ ]:
from chest import Chest
help(Chest)

##  Dblk Sparse IO Test 

In [ ]:
path='/ffffffffffff/tag/sd/0/dblk/byte'
handler, path_list = LocateNode(tag_tree, path)
type(handler)
from binascii import hexlify

In [ ]:
type(handler)

In [ ]:
handler.sparse

In [ ]:
for k in handler.sparse:
    print(k)

In [ ]:
attrs = handler.getattr(path_list, update=True)
attrs

In [ ]:
b = handler.read(path_list, 4*512, 0*512)
print(len(b), hexlify(b))

In [ ]:
handler.flush(path_list)

In [ ]:
handler.unlink(path_list)

## Test Chest Module

In [ ]:
STOP

In [ ]:
import os
from chest import Chest

In [ ]:
os.mkdir('echo')

In [ ]:
c=Chest(path='echo/1')

In [ ]:
c[100]='100'
c[200] = '200'

In [ ]:
for k in c: print(k)

In [ ]:
c.flush()

In [ ]:
c.clear()

In [ ]:
c.drop()

In [ ]:
help(Chest)

## Get Poll Count and Events (msg level)

In [ ]:
STOP

In [ ]:
def get_poll_info(radio, path, name):
    
    def _get_poll_msg(path, name):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.append(TagTlv(name))
        req_obj = TagHead(TagName(tlv_list))
        return req_obj

    req_msg = _get_poll_msg(path, name)
    err, payload = msg_exchange(radio, req_msg)
    if (err is None) or (err == tlv_errors.SUCCESS):
        size = payload2values(payload,
                                [tlv_types.SIZE,
                                ])[0]

        return size
    return None

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/poll'

In [ ]:
print('poll count', get_poll_info(radio, TAG_PATH, 'cnt'))
print('poll events', get_poll_info(radio, TAG_PATH, 'ev'))

## Write Dblk Note

In [ ]:
STOP

In [ ]:
path='/ffffffffffff/tag/sd/0/dblk/note'
handler, path_list = LocateNode(tag_tree, path)
type(handler)

In [ ]:
handler.write(path_list, 'this is a note', 0)

In [ ]:
handler.getattr(path_list, update=True)

## test locateNode

In [ ]:
STOP

In [ ]:
path='/ffffffffffff/tag/sd/0/dblk/note'
path='/'
handler, path_list = LocateNode(tag_tree, path)
type(handler)

In [ ]:
handler